In [1]:
from dotenv import load_dotenv
from pathlib import Path
from openai import AsyncAzureOpenAI
import os
from agentbyte.llm.auth import get_certificate_token_provider
from agentbyte.llm.azure_openai import AzureOpenAIChatCompletionClient

In [2]:
ENV_PATH = Path().absolute().parent / ".envs" / "local.env"
print(f"Loading environment variables from: {ENV_PATH}")

Loading environment variables from: /Users/alamshc/PythonProjects/AiEngineering/agentbyte/.envs/local.env


In [3]:
load_dotenv(ENV_PATH, override=True)

True

In [4]:
certificate_data = (
    os.environ["AZURE_CLIENT_CERTIFICATE_SECRET"].encode() 
    + b"\n" 
    + os.environ["AZURE_CLIENT_PRIVATE_KEY_SECRET"].encode()
)


In [5]:
token_provider = get_certificate_token_provider(
    tenant_id=os.environ["AZURE_TENANT_ID"],
    client_id=os.environ["AZURE_CLIENT_ID"],
    certificate_data=certificate_data,
    scope=os.environ["AZURE_COGNITIVE_SCOPE"]
)

In [6]:
client = AsyncAzureOpenAI(
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    azure_ad_token_provider=token_provider
)

In [7]:
azure_chat_client = AzureOpenAIChatCompletionClient(model="gpt-4.1", client=client)

In [8]:
from agentbyte.llm import SystemMessage, UserMessage

In [9]:
# Create conversation history
messages = [
    SystemMessage(content="You are a helpful AI assistant specialized in software engineering and Python development."),
    UserMessage(content="What are the key benefits of using async/await in Python?")
]

print("Conversation History:")
for i, msg in enumerate(messages, 1):
    print(f"{i}. {msg.role}: {msg.content[:60]}..." if len(msg.content) > 60 else f"{i}. {msg.role}: {msg.content}")

Conversation History:
1. MessageRole.SYSTEM: You are a helpful AI assistant specialized in software engin...
2. MessageRole.USER: What are the key benefits of using async/await in Python?


In [10]:
# Call the create method to get a response
response = await azure_chat_client.create(messages=messages)

print("\n" + "="*60)
print("Response from Azure OpenAI:")
print("="*60)
print(f"Content: {response.message.content}")
print(f"Role: {response.message.role}")
print(f"Model: {response.model}")
print(f"\nUsage Statistics:")
print(f"  Input tokens: {response.usage.tokens_input}")
print(f"  Output tokens: {response.usage.tokens_output}")
print(f"  Total tokens: {response.usage.total_tokens}")
if response.metadata:
    print(f"\nMetadata: {response.metadata}")


Response from Azure OpenAI:
Content: **Async/await** in Python is a way to write asynchronous, non-blocking code using the `async` and `await` keywords (introduced in Python 3.5). Here are the **key benefits**:

---

### 1. Improved Concurrency & Non-blocking I/O
You can handle many I/O-bound operations (networking, file I/O, database) concurrently—*without* using threads or processes. While one operation "waits" for data (e.g., network response), the event loop runs other tasks.

### 2. Simpler Code than Callbacks or Threads
The syntax with `async` and `await` is **much more readable and maintainable** than chaining callbacks (as in `Twisted`) or managing threads (and synchronization). Asynchronous code looks almost like regular, sequential code.

**Example:**
```python
async def fetch_data():
    data = await some_network_call()
    print(data)
```

### 3. Scalability for I/O-bound Work
Async/await shines in high-concurrency scenarios: web servers, bots, network tools, scraping, etc

In [12]:
# Test streaming feature
streaming_messages = [
    SystemMessage(content="You are a helpful AI assistant. Keep responses concise."),
    UserMessage(content="List 3 key Python features for async programming.")
]

print("\n" + "="*60)
print("Testing Streaming Response:")
print("="*60 + "\n")

# Collect full response while streaming
full_content = ""
chunk_count = 0

async for chunk in azure_chat_client.create_stream(messages=streaming_messages):
    chunk_count += 1
    print(chunk.content, end="", flush=True)
    full_content += chunk.content

print("\n\n" + "="*60)
print(f"Streaming Complete!")
print(f"  Chunks received: {chunk_count}")
print(f"  Total characters: {len(full_content)}")
print("="*60)


Testing Streaming Response:

1. **async/await syntax:** Enables writing asynchronous code that is easy to read and maintain.
2. **asyncio library:** Provides the main framework for writing single-threaded concurrent code using coroutines, event loops, tasks, and futures.
3. **Coroutines:** Special functions defined with `async def` that allow suspension and resumption of execution, enabling non-blocking I/O and concurrent operations.

Streaming Complete!
  Chunks received: 85
  Total characters: 408
